
# System Optimization - Damage Model
This is the main system optimization (MODOS) - Damage Model. It is part of a research project at the TU Delft on developing a tool which can help decision making early on in the design process. It uses the limited data availabe during conceptual design to show how design choices affect an area vulnerable to flooding and helps to optimize the system of flood risk reduction measures.

## Model Setup
The system optimization works by combining two models: The Optimization Model and the Damage Model. The Damage Model (this file) simulates the impact of a given storm on a vulnerable region. Also, it is possible to simulate the change of impact due to the implementation of a Flood Risk Reduction Stategy. This strategy can consist of a number of different flood risk reduction measures.

The Optimization Model is able to run the Damage Model numerous times, with different storms and different Flood Risk Reduction Strategies an analyze the outcome. With this data, it is possible to learn and predict how the region would react to particular design choices and show how the impact of one measure would change by changing the rest of the system. The Optimization Model is not included in this notebook.

## Damage Model
The core of the system optimzation is the Damage Model, which is primarily run from this script. It models the vulnerable region as a sequence of Protective Layers and Protected Areas. Here, the protective layers can (but do not have to) consist of flood defense structures, like levees or storm surge barrier. These can be placed on land, as well as in water. The protected areas are defined as areas where flood protective is not the primary land use. This is where the people live and work and will  therefore account for most of the damage. The damage follows from a calculation of the water levels in different parts of the region. 

The water levels, wave heights, failure probabilities and anything related to these variables are based on simplified hydraulic formulas. The boundary conditions at sea are based on externally run (ADCIRC) models. 

The Damage Model follows the path of the flood, as it starts at sea and runs toward the coast and further inland. The strength of the model is its ability to include multiple layers of defense. Also, it is possible to include a wide variety of flood risk reduction measures, from structural flood defenses to Nature-based Solutions or Spatial planning policies.

## How to use the Damage Model
Many functions in this model are based on definitions and calculations from other files. These files should be included in a folder "Library", which should be located in the same folder as the model itself. the required files are:

 - DamageModelCalculations.py 
 - DamageModelDefinitions.py

The model is still under construction. If you make any changes, please note the latest changes.
<br>
<br>
Author: E.C. van Berchum(TU Delft)


Version Damage Model: V0.1
Last Updated on June 28 (Erik):

 - imported to jupyter     


In [1]:
import sys
sys.path.append('./Packages')
%pwd

'D:\\ecvanberchum\\Surfdrive\\MODOS\\MODOS_model\\FLORES_main'

In [3]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

from Library.simulation_calculations_beira import run_hydraulic_calculation
from Library.simulation_definitions_beira import (Impact)

import Library.simulation_data_beira as data

%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as ss
import numpy as np
import seaborn as sns
from timeit import default_timer as timer
import csv

time_prep = timer()
basins_master = data.load_basins("Library/input_data/region_layout_basins.csv",
                            "Library/input_data/urban_development_scenarios.csv")  # Regional Layout
layers_master = data.load_layers("Library/input_data/region_layout_layers.csv", basins_master)
all_measures_master = data.load_measures("Library/input_data/flood_risk_reduction_measures.csv")
hydraulic_conditions_master = data.load_hydraulic_conditions("Library/input_data/hydraulic_boundary_conditions_surge_beira.csv",
                                                        "Library/input_data/hydraulic_boundary_conditions_rain_beira.csv")

damage_curves = data.load_damage_curves('Library/input_data/global_flood_depth-damage_functions__30102017.xlsx', 'AFRICA',
                                   'Mozambique', 'Object based', 1.30)
data.load_basin_borders(basins_master, "Library/input_data/region_layout_basin_borders.csv")
drainage_master = data.load_drainage_capacities(basins_master, 'Library/input_data/basins_drainage.csv', small_channel=6,
                                           mid_channel=10, large_channel=35, low_drain=0, mid_drain=2, high_drain=4)




# Define the model

This part shows all the main calculations done inside the model. The Galveston Bay Region is divided into two layers of flood protection (the Coastal Zone and the Inland Protective Layer) and two layers of Protected Area (1st Protected Area and 2nd Protected Area).

The current chosen output is shown after the 'return' command. This includes information on:
 - Expected damage costs as a result of a 1 / (RP_Storm) year event with the chosen strategy in place
 - Construction cost of the flood risk reduction measures that make up the strategy
 - Expected repair costs to flood risk reduction measures due to failure, weigted for probability of failure
 - Inland codes. these are the 16 possible inland scenarios, with probability and water level. 
     the codes vary from 1.1 to 4.4, where the first number is the coastal situation and the second the situation 
     at the inland barrier. the numbers mean: 1 = both barriers hold. 2 = land barrier fails. 3 = water barrier fails.
     4 = both barriers fail.
     
 - Coastal situation. These are the 4 possible outcomes of the situation in the coastal zone, with probability,
   water level, surge data at the coast and resulting surge inside the bay.
   
 - Hydraulic boundary conditions (hydr). This dictionary includes all hydraulic input parameters.
 

In [4]:
def flood_simulation_model(return_period_storm = 0,    # 0, 2, 5, 10, 50, 100
                             return_period_rain = 0,    # 0, 2, 5, 10, 50, 100
                             struc_measure_coast_1 = 'none',  # 'Heighten dunes east','Sand supplements east', None
                             struc_measure_coast_2 = 'none',  # 'Heighten dunes west','Floodwall west'', None
                             struc_measure_inland_1 = 'none',  # 'Heighten inland road', None
                             struc_measure_inland_2 = 'none',
                             drainage_measure_1 = 'none',  # 'Second phase drainage', None
                             drainage_measure_2 = 'none',   # 'Microdrainage, None
                             retention_measure_1 = 'none',  # 'East retention', None
                             retention_measure_2 = 'none',    # 'Maraza retention, None
                             emergency_measure_1 = 'none',  # 'Improve evacuation', None
                             emergency_measure_2 = 'none',    # 'Early warning system', None
                             h_struc_measure_coast_1 = 0,  # 8-12
                             h_struc_measure_coast_2 = 0,  # 8-12
                             h_struc_measure_inland_1 = 0,  # 7-10
                             h_struc_measure_inland_2 = 0,
                             input_scenario_climate = 'none',  # 'high','low', 'none'
                             input_scenario_development = 'none'  # 'high','low', 'none'
                            ):
        # added so we can add scenario information into the replicator
    if return_period_rain == 'INFO':
        scenario_info = str(input_scenario_climate) + ',' + str(input_scenario_development)
        return [scenario_info]*3
    
        # start of model, loads simulation-specific data
    hydraulic = data.get_hydraulic_boundary_conditions(hydraulic_conditions_master, return_period_storm, return_period_rain,
                                                  input_scenario_climate)
    region_layout = data.get_region_layout(basins_master, layers_master, input_scenario_development)
    strategy = data.get_strategy(all_measures_master, region_layout,
                            [struc_measure_coast_1, struc_measure_coast_2, struc_measure_inland_1, struc_measure_inland_2,
                             drainage_measure_1, drainage_measure_2, retention_measure_1, retention_measure_2,
                             emergency_measure_1, emergency_measure_2],
                            [h_struc_measure_coast_1, h_struc_measure_coast_2, h_struc_measure_inland_1,
                             h_struc_measure_inland_2])
    impact = Impact()

    # Builds and correctly names the scenarios
    for sequence in [1, 3]:
        region_layout.Layers[sequence].get_scenarios()
    strategy.get_list_scenarios(region_layout)

    # Hydraulic calculations, runs entire hydraulic simulation (pluvial and storm surge flooding)
    run_hydraulic_calculation(region_layout, hydraulic, strategy)

    strategy.get_probabilities(region_layout, hydraulic)

    # Calculate cost of construction and repair
    total_cost = strategy.get_construction_costs()

    # Impact calculations, calculates expected damages and exposed population per basin and in total
    impact.run_impact_calculations(region_layout, strategy, damage_curves)
    expected_damage = impact.TotalExpectedDamage
    construction_cost = total_cost
    affected_population = impact.TotalExpectedExposedPop
    
    return expected_damage, construction_cost, affected_population

# Run the model without barrier

The following command runs the model. here you can choose the input parameters, consisting of different flood risk reduction measures, heights of the flood risk reduction measures and the return period of the storm.

Also, the output list needs to allign with the 'return'-statement in the defined function above. 

At this point, no barrier are chosen. all scenarios look the same, this results in the maximum damage for this return period of storm

In [10]:
start = timer()

[expected_damage, construction_cost, affected_population] = flood_simulation_model(
    return_period_storm = 0,
    return_period_rain = 100,
    struc_measure_coast_1 = 'none',  # 'Heighten dunes east','Sand supplements east', None
    struc_measure_coast_2 = 'none',  # 'Heighten dunes west','Floodwall west'', None
    struc_measure_inland_1 = 'none',  # 'Heighten inland road', None
    struc_measure_inland_2 = 'none',                   # None
    drainage_measure_1 = 'none',  # 'Second phase drainage', None
    drainage_measure_2 = 'none',  # 'Microdrainage, None
    retention_measure_1 = 'none',  # 'East retention', None
    retention_measure_2 = 'none',   # 'Chota retention', None
    emergency_measure_1 = 'none',  # 'Improve evacuation', None
    emergency_measure_2 = 'none',   # 'Early warning system', None
    h_struc_measure_coast_1 = 8.2731906,  # 8-12
    h_struc_measure_coast_2 = 11.872320,  # 8-12
    h_struc_measure_inland_1 = 11.3344123,  # 7-10
    h_struc_measure_inland_2 = 0,     # 0
    input_scenario_climate = 'high',  # 'high','low', 'none'
    input_scenario_development = 'low'  # 'high','low', 'none'
    )
end = timer()

print('computation time is {} seconds'.format(end - start))
print('total damage is: ', expected_damage)
print('total construction cost is: ', construction_cost)
print('total affected population is: ', affected_population)

computation time is 11.114223437045894 seconds
total damage is:  83422035.82623976
total construction cost is:  0
total affected population is:  304563.11514265556


In [9]:
start = timer()

[expected_damage, construction_cost, affected_population] = flood_simulation_model(
    return_period_storm = 0,
    return_period_rain = 100,
    struc_measure_coast_1 = 'none',  # 'Heighten dunes east','Sand supplements east', None
    struc_measure_coast_2 = 'none',  # 'Heighten dunes west','Floodwall west'', None
    struc_measure_inland_1 = 'none',  # 'Heighten inland road', None
    struc_measure_inland_2 = 'none',                   # None
    drainage_measure_1 ='Second_phase_drainage',  # 'Second phase drainage', None
    drainage_measure_2 = 'none',  # 'Microdrainage, None
    retention_measure_1 = 'East_retention',  # 'East retention', None
    retention_measure_2 = 'none',   # 'Chota retention', None
    emergency_measure_1 = 'Improve_evacuation',  # 'Improve evacuation', None
    emergency_measure_2 = 'none',   # 'Early warning system', None
    h_struc_measure_coast_1 = 8.2731906,  # 8-12
    h_struc_measure_coast_2 = 11.872320,  # 8-12
    h_struc_measure_inland_1 = 11.3344123,  # 7-10
    h_struc_measure_inland_2 = 0,     # 0
    input_scenario_climate = 'high',  # 'high','low', 'none'
    input_scenario_development = 'low'  # 'high','low', 'none'
    )
end = timer()

print('computation time is {} seconds'.format(end - start))
print('total damage is: ', expected_damage)
print('total construction cost is: ', construction_cost)
print('total affected population is: ', affected_population)

computation time is 11.379789489859093 seconds
total damage is:  80868072.48283014
total construction cost is:  18.5
total affected population is:  175213.25049992948
